*Copyright (c) Cornac Authors. All rights reserved.*

*Licensed under the Apache 2.0 License.*

# Visual Bayesian Personalizer Ranking with Text Data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/PreferredAI/cornac/blob/master/tutorials/vbpr_text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/PreferredAI/cornac/blob/master/tutorials/vbpr_text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

We would like to use [Visual Bayesian Personalizer Ranking (VBPR)](https://arxiv.org/pdf/1510.01784.pdf), it makes use of pre-trained visual features extracted from CNN. However, our data of interest [MovieLens dataset](https://grouplens.org/datasets/movielens/) does not come with visual information, but instead it contains text movie plots. In this tutorial, we will employ Conac's modality infrastructures to easily utilize VBPR to leverage item text content.

## Setup

In [ ]:
# install Cornac
!pip3 install numpy cornac
# install PyTorch for VBPR model
!pip3 install torch==1.0.0

import cornac
from cornac.data import Reader
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality, ImageModality
from cornac.data.text import BaseTokenizer

The directory '/home/tuantruong/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/tuantruong/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 18.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
The directory '/home/tuantruong/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/tuantruong/.cache/pip' or its parent directory is not owned by the current user 

## Prepare data
Here we use the MovieLens 100K dataset which is already accessible from Cornac. Hence, we can simply load movie plots and the rating data.

In [ ]:
plots, movie_ids = movielens.load_plot()

# movies without plots are filtered out by `cornac.data.Reader`
ml_100k = movielens.load_100k(reader=Reader(item_set=movie_ids)) 

## Cross modality

To get representations from text data, we build a `TextModality` using text corpus with coressponding ids as the input. We also need to supply a `Tokenizer` for text splitting, in this case the tokens in text plots are seperated by `\tab` character. We limit the maximum size of vocabulary to 5000, which also means the size of representations (e.g., bag-of-words vectors) can not go beyond that limit.

In [ ]:
item_text_modality = TextModality(corpus=plots, ids=movie_ids, 
                                  tokenizer=BaseTokenizer(sep='\t', stop_words='english'),
                                  max_vocab=5000, max_doc_freq=0.5).build()

After building the `TextModality`, we can access text representations through provided APIs and attributes of the object. In this case, we take the word-count matrix as item representations then use it as input to construct an `ImageModality`. In other words, item visual representations will be substituted by bag-of-words representations which are used by `VBPR` model. 

In [ ]:
features = item_text_modality.count_matrix.A
item_image_modality = ImageModality(features=features, ids=movie_ids)

In Cornac, image recommendation models always work with `ImageModality`, similarly for other modalities. That design makes it consistent with models' original assumptions, thus it's easy to integrate models into the library without any confusion of which modality to use.

## Experiment

We employ `RatioSplit` evaluation method to split the rating data. The `item_image_modality` is also supplied here for later usage of model training.

In [ ]:
ratio_split = RatioSplit(data=ml_100k, test_size=0.2,
                         item_image=item_image_modality,
                         exclude_unknowns=True, 
                         verbose=True, seed=123)

VBPR model is supported inside Cornac. Note that the implementation of VBPR uses [PyTorch](https://pytorch.org/), thus we need to install that dependency in order to run the model (done at the beginning). List of implemented models with their additional dependencies is carefully documented [here](https://github.com/PreferredAI/cornac#models).
We also use [BPR](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf) model as a baseline to see the effectiveness of the text auxiliary data.

In [ ]:
vbpr = cornac.models.VBPR(k=10, k2=10, n_epochs=100, batch_size=50, learning_rate=0.001,
                          lambda_w=1, lambda_b=0.01, lambda_e=0.0, use_gpu=True, seed=123)

bpr = cornac.models.BPR(k=20, max_iter=100, learning_rate=0.01, lambda_reg=0.001, seed=123)

In [ ]:
auc = cornac.metrics.AUC()
rec_50 = cornac.metrics.Recall(k=50)

In [ ]:
cornac.Experiment(eval_method=ratio_split,
                  models=[bpr, vbpr],
                  metrics=[auc, rec_50]).run()

Results after running the experiment:

<pre>
     |    AUC | Recall@50 | Train (s) | Test (s)
---- + ------ + --------- + --------- + --------
BPR  | 0.8204 |    0.2236 |    0.2269 |   0.5853
VBPR | 0.8587 |    0.3012 |  199.3364 |   0.8134
</pre>